# Multi-Role Workflow with Specialized Agents

## Overview

This notebook demonstrates a structured multi-agent workflow using five specialized agents coordinated by an intelligent manager. The workflow follows a systematic pattern:

```
┌─────────────────────────────────────────────────────────┐
│                   Task Input                            │
└─────────────────┬───────────────────────────────────────┘
                  │
                  ▼
          ┌───────────────┐
          │  Executor     │ ◄─── Carries out reasoning-heavy
          │ (gpt-5-mini)  │      steps and delegates to tools
          └───────┬───────┘
                  │
                  ▼
          ┌───────────────┐
          │  Coder        │ ◄─── Writes and executes code
          │ (gpt-5-mini)  │      for computation and analysis
          └───────┬───────┘
                  │
                  ▼
          ┌───────────────┐
          │  Verifier     │ ◄─── Inspects outputs, confirms
          │ (gpt-5-mini)  │      requirements are satisfied
          └───────┬───────┘
                  │
                  ▼
          ┌───────────────┐
          │  Generator    │ ◄─── Assembles final response
          │ (gpt-5-mini)  │      with verified outputs
          └───────────────┘
```

## How It Works

The `MagenticBuilder` creates a workflow with a `StandardMagenticManager` that:

1. **Reads agent descriptions** to understand each agent's capabilities
2. **Creates dynamic plans** based on the task requirements
3. **Selects appropriate agents** for each step of the plan
4. **Tracks progress** and adapts when agents stall or plans need revision
5. **Manages completion** when the task is successfully resolved

## Model Assignments

Each agent uses the same model tier for consistency:

- **Planner (gpt-5-mini)**: Provides advanced reasoning for decomposing complex tasks
- **Executor (gpt-5-mini)**: Handles reasoning-heavy execution steps
- **Coder (gpt-5-mini + code interpreter)**: Generates and runs code via the hosted interpreter
- **Verifier (gpt-5-mini)**: Validates outputs and ensures quality
- **Generator (gpt-5-mini)**: Synthesizes verified outputs into a polished response

## Prerequisites

Set these environment variables:

```bash
export OPENAI_API_KEY="your-api-key-here"
# Optional: Specify endpoints if using Azure OpenAI
# export AZURE_OPENAI_ENDPOINT="your-endpoint"
# export AZURE_OPENAI_API_KEY="your-key"
```

In [ ]:
# Copyright (c) Microsoft. All rights reserved.
## Prerequisites

In [ ]:
export OPENAI_API_KEY="your-api-key-here"
export OPENAI_BASE_URL="https://api.openai.com/v1"
# Optional: Specify endpoints if using Azure OpenAI
# export AZURE_OPENAI_ENDPOINT="your-endpoint"
# export AZURE_OPENAI_API_KEY="your-key"


## Step 1: Import Required Modules

We import the core agent framework components:
- `ChatAgent`: Base agent class for creating specialized agents
- `MagenticBuilder`: Workflow builder for multi-agent orchestration
- `HostedCodeInterpreterTool`: Tool for code execution capabilities
- Event types for monitoring workflow execution
- OpenAI client implementations for different model APIs

In [1]:
import logging

from agent_framework import (
    ChatAgent,
    HostedCodeInterpreterTool,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticBuilder,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
    WorkflowOutputEvent,
)
from agent_framework.openai import OpenAIResponsesClient
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Step 2: Configure Debug Logging

Enable debug logging to observe the manager's decision-making process:

- **INFO level**: Shows high-level workflow progress
- **DEBUG level**: Reveals manager's agent selection logic, plan creation, and progress tracking

You can change this to `logging.INFO` for cleaner output, or `logging.DEBUG` to see internal orchestration decisions.

In [2]:
# Set to DEBUG to see manager's decision-making, INFO for cleaner output
logging.basicConfig(level=logging.INFO, force=True)
logger = logging.getLogger(__name__)

print("✓ Logging configured at INFO level (change to DEBUG for detailed internal logs)")

✓ Logging configured at INFO level (change to DEBUG for detailed internal logs)


## Step 3: Define Role Prompts

Each agent receives specialized instructions that define its role in the workflow. These prompts guide the agent's behavior and help the manager understand when to invoke each agent.

In [3]:
EXECUTOR_PROMPT = """You are the executor module. Carry out the active instruction from the
manager or planner. Execute reasoning-heavy steps, delegate to registered tools when needed,
and produce clear artefacts or status updates. If a tool is required, call it explicitly and
then explain the outcome."""

VERIFIER_PROMPT = """You are the verifier module. Inspect the current state, outputs, and
assumptions. Confirm whether the work satisfies requirements, highlight defects or missing
information, and suggest concrete follow-up actions."""

GENERATOR_PROMPT = """You are the generator module. Assemble the final response for the
user. Incorporate verified outputs, cite supporting evidence when available, and ensure the
result addresses the original request without leaking internal reasoning unless explicitly
requested."""

## Step 5: Configure the Executor Agent

**Role**: Carries out reasoning-heavy steps and delegates to tools when computation is needed.

**Model**: `gpt-5-mini` - Balances capability and cost for general execution tasks.

**When invoked**: The manager selects the Executor for steps requiring reasoning, analysis, or coordination between other agents.

The Executor acts as the "general worker" that handles steps not requiring specialized capabilities.

In [4]:
executor_agent = ChatAgent(
    name="ExecutorAgent",
    description="Executes reasoning-heavy tasks and coordinates work between specialized agents.",
    instructions=EXECUTOR_PROMPT,
    chat_client=OpenAIResponsesClient(
        model_id="gpt-5-mini",
        reasoning_effort="medium",
        store=True,
        temperature=0.7,
        max_tokens=4096,
    ),
)

print("✓ Executor Agent configured with gpt-5-mini and enhanced parameters")

✓ Executor Agent configured with gpt-5-mini and enhanced parameters


## Step 6: Configure the Coder Agent

**Role**: Writes and executes code for data processing, calculations, and analysis.

**Model**: `gpt-5-mini` paired with the hosted code interpreter for program execution.

**Tools**: `HostedCodeInterpreterTool()` - Enables code execution in a hosted sandbox environment.

**When invoked**: The manager selects the Coder when computational analysis, data processing, or numerical calculations are needed.

Note: The code execution happens in OpenAI's hosted environment, not locally. Results are returned as part of the agent's response.

In [5]:
coder_agent = ChatAgent(
    name="CoderAgent",
    description="Writes and executes code to perform calculations, data analysis, and computational tasks.",
    instructions="You solve questions using code. Write clear, well-documented code and provide detailed analysis of computation results.",
    chat_client=OpenAIResponsesClient(
        model_id="gpt-5-mini",
        reasoning_effort="high",  # Higher reasoning for code generation
        store=True,
        temperature=0.3,  # Lower temperature for more deterministic code
        max_tokens=8192,  # More tokens for code + explanations
    ),
    tools=HostedCodeInterpreterTool(),
)

print("✓ Coder Agent configured with gpt-5-codex and enhanced parameters")

✓ Coder Agent configured with gpt-5-codex and enhanced parameters


## Step 7: Configure the Verifier Agent

**Role**: Validates outputs, checks for defects, and ensures requirements are satisfied.

**Model**: `gpt-5-mini` - Provides reliable reasoning to perform thorough quality checks.

**When invoked**: The manager calls the Verifier after execution steps to validate correctness, completeness, and adherence to requirements.

The Verifier acts as a quality gate, catching errors before final response generation.

In [6]:
verifier_agent = ChatAgent(
    name="VerifierAgent",
    description="Validates outputs, checks assumptions, and confirms work meets requirements.",
    instructions=VERIFIER_PROMPT,
    chat_client=OpenAIResponsesClient(
        model_id="gpt-5-mini",
        reasoning_effort="high",  # High reasoning for thorough validation
        store=True,
        temperature=0.5,  # Balanced for analytical verification
        max_tokens=4096,
    ),
)

print("✓ Verifier Agent configured with gpt-5-mini and enhanced parameters")

✓ Verifier Agent configured with gpt-5-mini and enhanced parameters


## Step 8: Configure the Generator Agent

**Role**: Assembles the final user-facing response by synthesizing verified outputs.

**Model**: `gpt-5-mini` - Cost-efficient model suitable for synthesis and formatting tasks.

**When invoked**: The manager selects the Generator as the final step to create a polished response for the user.

The Generator ensures the final output is clear, comprehensive, and addresses the original request without exposing internal workflow details.

In [7]:
generator_agent = ChatAgent(
    name="GeneratorAgent",
    description="Synthesizes final responses by incorporating verified outputs and supporting evidence.",
    instructions=GENERATOR_PROMPT,
    chat_client=OpenAIResponsesClient(
        model_id="gpt-5-mini",
        reasoning_effort="low",  # Lower reasoning for synthesis tasks
        store=True,
        temperature=0.8,  # Higher temperature for creative synthesis
        max_tokens=6144,  # More tokens for comprehensive responses
    ),
)

print("✓ Generator Agent configured with gpt-5-mini and enhanced parameters")

✓ Generator Agent configured with gpt-5-mini and enhanced parameters


## Understanding OpenAI Responses API Parameters

The `OpenAIResponsesClient` accepts several parameters to fine-tune agent behavior:

### Key Parameters

- **`reasoning_effort`**: Controls how much computational effort the model uses for reasoning
  - `"low"`: Faster responses, suitable for simple tasks or synthesis
  - `"medium"`: Balanced performance (default for most agents)
  - `"high"`: Maximum reasoning depth for complex analysis, verification, or planning

- **`store`**: Boolean flag to enable conversation storage
  - `True`: Stores conversations for potential learning and improvement
  - `False`: No storage (default)
  - Useful for building persistent context across sessions

- **`temperature`**: Controls randomness in responses (0.0 - 2.0)
  - `0.0-0.3`: Deterministic, focused (ideal for code generation)
  - `0.4-0.7`: Balanced creativity and consistency (general tasks)
  - `0.8-1.0`: More creative and varied (synthesis, brainstorming)
  - `>1.0`: Highly creative but potentially inconsistent

- **`max_tokens`**: Maximum length of the response
  - Adjust based on expected output length
  - Code agents may need more tokens (8192+)
  - Simple agents can use fewer (2048-4096)

- **`top_p`**: Nucleus sampling (0.0 - 1.0)
  - Controls diversity via cumulative probability
  - Lower values = more focused responses

- **`frequency_penalty`**: Reduces repetition (-2.0 to 2.0)
  - Positive values discourage repeating the same words

- **`presence_penalty`**: Encourages topic diversity (-2.0 to 2.0)
  - Positive values encourage exploring new topics

### Agent-Specific Tuning in This Notebook

- **Planner**: Medium reasoning, moderate temperature (balanced planning)
- **Executor**: Medium reasoning, moderate temperature (general execution)
- **Coder**: High reasoning, low temperature (precise code generation)
- **Verifier**: High reasoning, moderate temperature (thorough validation)
- **Generator**: Low reasoning, higher temperature (creative synthesis)
- **Manager**: High reasoning, moderate temperature (strategic orchestration)

## Step 9: Understanding the MagenticBuilder Pattern

The `MagenticBuilder` constructs a workflow with intelligent orchestration:

1. **`.participants()`** - Registers all agents with the workflow. Each agent's `name` and `description` help the manager understand its capabilities.

2. **`.with_standard_manager()`** - Configures the orchestration manager with:
   - **chat_client**: LLM used by the manager for decision-making
   - **max_round_count**: Maximum conversation turns (prevents infinite loops)
   - **max_stall_count**: How many times agents can be unproductive before replanning
   - **max_reset_count**: How many times workflow can reset from scratch

The manager uses these parameters to adaptively coordinate agents throughout the workflow lifecycle.

## Step 10: Build the Workflow

Now we construct the workflow by:

1. Creating a `MagenticBuilder` instance
2. Registering all five agents as participants
3. Configuring the standard manager with appropriate limits
4. Building the final workflow object

The manager will read each agent's `description` field to understand when to invoke them during execution.

In [8]:
print("\nBuilding Magentic Workflow with 5 specialized agents...")

workflow = (
    MagenticBuilder()
    .participants(
        coder=coder_agent,
        verifier=verifier_agent,
        generator=generator_agent,
    )
    .with_standard_manager(
        chat_client=OpenAIResponsesClient(
            model_id="gpt-5-mini",
            reasoning_effort="high",  # Manager needs high reasoning for orchestration
            store=True,
            temperature=0.6,
            max_tokens=8192,  # Manager needs more tokens for planning
        ),
        max_round_count=6,  # Allow enough rounds for 5-agent coordination
        max_stall_count=3,   # Replan if agents stall 3 times
        max_reset_count=2,   # Allow 2 full workflow resets if needed
    )
    .build()
)

print("✓ Workflow built successfully!")
print(f"  - Registered agents: {len([coder_agent, verifier_agent, generator_agent])}")
print("  - Manager model: gpt-5-mini with enhanced parameters")
print("  - Max rounds: 6")
print("\n⚠️  Note: 5-agent workflows using gpt-5-mini can take a few minutes to complete.")

INFO:agent_framework._workflows._magentic:Building Magentic workflow with 3 participants



Building Magentic Workflow with 5 specialized agents...
✓ Workflow built successfully!
  - Registered agents: 3
  - Manager model: gpt-5-mini with enhanced parameters
  - Max rounds: 6

⚠️  Note: 5-agent workflows using gpt-5-mini can take a few minutes to complete.


## Step 11: Define the Task

We'll use a complex task that exercises all five agents:

- **Planner**: Decomposes the multi-part analysis
- **Executor**: Coordinates research and execution
- **Coder**: Writes code to calculate energy consumption and CO2 emissions
- **Verifier**: Validates calculations and assumptions
- **Generator**: Synthesizes findings into a comprehensive report

This task requires data analysis, computational verification, and structured output—ideal for demonstrating the multi-role workflow.

In [9]:
task = (
    "I'm build a ai system that help reasoning and problem parsing capabilities. "
)

print("\n" + "=" * 80)
print("TASK:")
print("=" * 80)
print(task)
print("=" * 80)


TASK:
I'm build a ai system that help reasoning and problem parsing capabilities. 


## Step 12: Understanding Workflow Events

The workflow emits several event types during execution:

- **`MagenticOrchestratorMessageEvent`**: Manager's planning, agent selection, and coordination messages
- **`MagenticAgentDeltaEvent`**: Streaming text chunks from agents (real-time output)
- **`MagenticAgentMessageEvent`**: Complete agent messages after streaming finishes
- **`MagenticFinalResultEvent`**: The workflow's final result when complete
- **`WorkflowOutputEvent`**: Structured output data from the workflow

By handling these events, we can observe:
- Which agent the manager selects for each step
- The manager's reasoning for agent selection
- Real-time progress as agents work
- Final results and completion status

## Step 13: Execute the Workflow with Streaming

Now we'll execute the workflow and observe the orchestration in action:

- Watch the manager select appropriate agents for each step
- See streaming output as agents work
- Observe the coordination between planning, execution, coding, verification, and generation

**Note**: This may take several minutes depending on task complexity and API response times.

In [10]:
import time
from datetime import datetime

print("\nStarting workflow execution...")
print(f"Started at: {datetime.now().strftime('%H:%M:%S')}")
print("⏳ This may take 1-3 minutes for complex tasks...\n")

last_stream_agent_id: str | None = None
stream_line_open: bool = False
final_output: str | None = None
event_count = 0
start_time = time.time()

try:
    async for event in workflow.run_stream(task):
        event_count += 1
        if event_count % 10 == 0:
            elapsed = time.time() - start_time
            print(f"\n[Progress: {event_count} events, {elapsed:.1f}s elapsed]\n", flush=True)

        if isinstance(event, MagenticOrchestratorMessageEvent):
            print(f"\n[ORCHESTRATOR:{event.kind}]\n")
            print(f"{getattr(event.message, 'text', '')}\n")
            print("-" * 80)

        elif isinstance(event, MagenticAgentDeltaEvent):
            if last_stream_agent_id != event.agent_id or not stream_line_open:
                if stream_line_open:
                    print()
                print(f"\n[STREAMING:{event.agent_id}]: ", end="", flush=True)
                last_stream_agent_id = event.agent_id
                stream_line_open = False
            if event.text:
                print(event.text, end="", flush=True)

        elif isinstance(event, MagenticAgentMessageEvent):
            if stream_line_open:
                print(" ✓")
                stream_line_open = False
            msg = event.message
            if msg is not None:
                response_text = (msg.text or "").replace("\n", " ")
                display_text = response_text[:200] + "..." if len(response_text) > 200 else response_text
                print(f"\n[AGENT:{event.agent_id}] {msg.role.value}")
                print(f"  {display_text}\n")
                print("-" * 80)

        elif isinstance(event, MagenticFinalResultEvent):
            if stream_line_open:
                print()
                stream_line_open = False

            print("\n" + "=" * 80)
            print("FINAL RESULT:")
            print("=" * 80)
            print("\n✓ Workflow completed successfully!\n")

            if event.message is not None:
                print(event.message.text)

            if final_output is not None:
                print("\nWorkflow Output Data:")
                print(final_output)

            print("=" * 80)

        elif isinstance(event, WorkflowOutputEvent):
            final_output = str(event.data) if event.data is not None else None

except Exception as exc:
    if stream_line_open:
        print()
    stream_line_open = False
    print(f"\nWorkflow execution failed: {exc}")

finally:
    if stream_line_open:
        print()

INFO:agent_framework._workflows._magentic:Magentic Orchestrator: Received start message
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'user'}, 'contents': [{'type': 'text', 'text': "I'm build a ai system that help reasoning and problem parsing capabilities. "}], 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'user'}, 'contents': [{'type': 'text', 'text': 'Below I will present you a request.\n\nBefore we begin addressing the request, please answer the following pre-survey to the best of your ability.\nKeep in mind that you are Ken Jennings-level with trivia, and Mensa-level with puzzles, so there should be\na deep well to draw from.\n\nHere is the request:\n\nI\'m build a ai system that help reasoning and problem parsing capabilities. \n\nHere is the pre-survey:\n\n    1. Please list any specific facts or figures that are GIVEN in the request itself. It is possible that\n       there are none


Starting workflow execution...
Started at: 13:30:25
⏳ This may take 1-3 minutes for complex tasks...



INFO:httpx:HTTP Request: POST https://fleetw7ot.openai.azure.com/openai/v1/responses "HTTP/1.1 200 OK"
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '1. GIVEN OR VERIFIED FACTS\n- You are building an AI system intended to help with reasoning and problem parsing capabilities.\n- The request does not include constraints, target domains, datasets, model choices, performance targets, or deployment requirements.\n\n2. FACTS TO LOOK UP\n- State‑of‑the‑art models and leaderboards for reasoning tasks (e.g., GPT‑4, PaLM, LLaMA family, open and closed models): check arXiv.org, Papers With Code (paperswithcode.com), and recent conference proceedings (NeurIPS, ACL, ICLR).\n- Relevant benchmarks and leaderboards (and their exact task definitions and metrics): GSM8K, MATH, MMLU, BIG‑Bench, ARC, DROP, AQuA, HumanEval — sources: Papers With Code pages for each dataset, the original dataset GitHub repos, and leaderboa


[ORCHESTRATOR:user_task]

I'm build a ai system that help reasoning and problem parsing capabilities. 

--------------------------------------------------------------------------------

[ORCHESTRATOR:task_ledger]


We are working to address the following user request:

I'm build a ai system that help reasoning and problem parsing capabilities. 


To answer this request we have assembled the following team:

- coder: Writes and executes code to perform calculations, data analysis, and computational tasks.
- verifier: Validates outputs, checks assumptions, and confirms work meets requirements.
- generator: Synthesizes final responses by incorporating verified outputs and supporting evidence.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- You are building an AI system intended to help with reasoning and problem parsing capabilities.
- The request does not include constraints, target domains, datasets, model choices, performance targets, or deployment requireme

INFO:httpx:HTTP Request: POST https://fleetw7ot.openai.azure.com/openai/v1/responses "HTTP/1.1 200 OK"


...


[Progress: 15170 events, 574.1s elapsed]


[STREAMING:verifier]: Summary
[STREAMING:verifier]:  /
[STREAMING:verifier]:  next
[STREAMING:verifier]:  step
[STREAMING:verifier]: 

[STREAMING:verifier]: -
[STREAMING:verifier]:  I
[STREAMING:verifier]:  cannot
[STREAMING:verifier]:  currently
[STREAMING:verifier]:  access
[Progress: 15180 events, 574.1s elapsed]


[STREAMING:verifier]:  the
[STREAMING:verifier]:  repository
[STREAMING:verifier]:  or
[STREAMING:verifier]:  the
[STREAMING:verifier]:  two
[STREAMING:verifier]:  files
[STREAMING:verifier]:  you
[STREAMING:verifier]:  named
[STREAMING:verifier]: .
[STREAMING:verifier]:  Please
[Progress: 15190 events, 574.2s elapsed]


[STREAMING:verifier]:  paste
[STREAMING:verifier]:  the
[STREAMING:verifier]:  contents
[STREAMING:verifier]:  of
[STREAMING:verifier]:  evaluation
[STREAMING:verifier]: _spec
[STREAMING:verifier]: .md
[STREAMING:verifier]:  and
[STREAMING:verifier]:  project
[STREAMING:verifier]: /m
[Progress: 15200 events, 574

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': 'Summary / next step\n- I cannot currently access the repository or the two files you named. Please paste the contents of evaluation_spec.md and project/mvp_outputs/human_eval_pilot_50.jsonl (or attach them). Once I have them I will (within 48 hours) either: (a) produce finalized evaluation_spec.md and the human-eval rubric/JSONL-approved markups, or (b) mark “approved as-is” if they already meet the acceptance gates below.\n- Meanwhile, below I provide a concrete, fully-specified recommended evaluation spec, numeric thresholds, rubric, labeling logistics, sandboxing/safety checks, automated harness checks, and a go/no-go gate. Treat these as the default changes I will apply to evaluation_spec.md unless you instruct otherwise or the files contain conflicting constraints.\n\n1) Automatic metrics and numeric thresholds (concrete numbers)\n- Overall approach:


[AGENT:verifier] assistant
  Summary / next step - I cannot currently access the repository or the two files you named. Please paste the contents of evaluation_spec.md and project/mvp_outputs/human_eval_pilot_50.jsonl (or attach ...

--------------------------------------------------------------------------------

[Progress: 18470 events, 629.4s elapsed]



INFO:agent_framework._workflows._runner:Completed superstep 12
INFO:agent_framework._workflows._runner:Workflow completed after 12 supersteps



FINAL RESULT:

✓ Workflow completed successfully!

Summary / next step
- I cannot currently access the repository or the two files you named. Please paste the contents of evaluation_spec.md and project/mvp_outputs/human_eval_pilot_50.jsonl (or attach them). Once I have them I will (within 48 hours) either: (a) produce finalized evaluation_spec.md and the human-eval rubric/JSONL-approved markups, or (b) mark “approved as-is” if they already meet the acceptance gates below.
- Meanwhile, below I provide a concrete, fully-specified recommended evaluation spec, numeric thresholds, rubric, labeling logistics, sandboxing/safety checks, automated harness checks, and a go/no-go gate. Treat these as the default changes I will apply to evaluation_spec.md unless you instruct otherwise or the files contain conflicting constraints.

1) Automatic metrics and numeric thresholds (concrete numbers)
- Overall approach: report per-task-type metrics and an aggregate. Separate task types: closed-answer cla

## Step 14: Using `.as_agent()` for Composition

The workflow can be wrapped as a reusable agent using `.as_agent()`. This allows:

- **Composition**: Use this workflow as a participant in larger workflows
- **Reusability**: Call the same workflow multiple times with different tasks
- **Transcript access**: Get structured message history from the workflow execution

This is powerful for building hierarchical agent systems where complex workflows become modular components.

In [ ]:
print("\n" + "=" * 80)
print("Demonstrating .as_agent() wrapper pattern...")
print("=" * 80)

# Wrap the workflow as an agent
workflow_agent = workflow.as_agent(name="MultiRoleWorkflowAgent")

# Execute through the agent interface
agent_result = await workflow_agent.run(task)

# Display the transcript
if agent_result.messages:
    print("\n===== Workflow Transcript =====\n")
    for i, msg in enumerate(agent_result.messages, start=1):
        role_value = getattr(msg.role, "value", msg.role)
        speaker = msg.author_name or role_value
        message_preview = (msg.text or "")[:150] + "..." if len(msg.text or "") > 150 else (msg.text or "")
        print(f"{'-' * 80}")
        print(f"Message {i:02d} [{speaker}]")
        print(f"{message_preview}")
    print(f"{'-' * 80}")
    print(f"\n✓ Total messages in transcript: {len(agent_result.messages)}")
else:
    print("No messages in transcript.")

## Optional: Customize Manager Parameters

You can experiment with different manager configurations to see how they affect workflow behavior:

- **Increase `max_round_count`** for more complex tasks requiring longer conversations
- **Decrease `max_stall_count`** to trigger replanning faster when agents aren't making progress
- **Add custom manager instructions** to guide orchestration behavior

Uncomment and run the cell below to try a different configuration.

In [ ]:
# # Example: Build workflow with custom manager configuration
# custom_workflow = (
#     MagenticBuilder()
#     .participants(
#         planner=planner_agent,
#         executor=executor_agent,
#         coder=coder_agent,
#         verifier=verifier_agent,
#         generator=generator_agent,
#     )
#     .with_standard_manager(
#         chat_client=OpenAIChatClient(model_id="gpt-5-mini"),
#         max_round_count=20,  # More rounds for complex tasks
#         max_stall_count=2,   # Faster replanning
#         max_reset_count=1,   # Fewer resets
#         instructions="Focus on efficiency. Always verify calculations before generating final outputs.",
#     )
#     .build()
# )
#
# print("✓ Custom workflow configured with modified parameters")

## Optional: Change Logging Level

Switch between logging levels to control output verbosity:

- **`DEBUG`**: See all manager decisions, agent selections, and internal state changes
- **`INFO`**: See workflow progress and key events
- **`WARNING`**: See only warnings and errors

Run the cell below to change the logging level, then re-execute the workflow cells above.

In [ ]:
# Change logging level (re-run workflow cells after changing this)
# logging.basicConfig(level=logging.DEBUG, force=True)  # Detailed internal logs
logging.basicConfig(level=logging.INFO, force=True)    # Clean progress logs
# logging.basicConfig(level=logging.WARNING, force=True)  # Warnings only

print("✓ Logging level updated")

## Troubleshooting Common Issues

### API Key Not Found
```
Error: OpenAI API key not found
```
**Solution**: Set the `OPENAI_API_KEY` environment variable before running the notebook.

### Model Not Available
```
Error: Model 'gpt-5-mini' does not exist
```
**Solution**: The notebook defaults to `gpt-5-mini` for every role. If this model is unavailable in your account, choose an accessible alternative and update the agent constructors. Common substitutes include:
- `gpt-4o` for high-quality reasoning
- `gpt-4o-mini` for lower latency and cost
- `gpt-4o-reasoning` or other reasoning-tier models if enabled for your workspace

### Rate Limits
```
Error: Rate limit exceeded
```
**Solution**: The workflow makes multiple API calls. If you hit rate limits:
- Use lower-tier models (e.g., `gpt-4o-mini` instead of `gpt-5-mini`)
- Reduce `max_round_count` to limit conversation length
- Add delays between workflow executions

### Workflow Stalls or Times Out
```
Warning: Max stall count reached
```
**Solution**: The manager detected agents aren't making progress:
- Simplify the task
- Adjust `max_stall_count` to allow more attempts
- Check agent instructions for clarity
- Review debug logs to see which agent is stalling

### Code Execution Fails
```
Error: Code execution failed
```
**Solution**: `HostedCodeInterpreterTool` requires:
- OpenAI Responses API or Azure OpenAI with code interpreter enabled
- Cannot execute arbitrary local code (runs in OpenAI's sandbox)
- Some operations (file I/O, network access) may be restricted

## Next Steps

Experiment with this workflow by:

1. **Modifying agent instructions** - Change role prompts to adjust behavior
2. **Adding new agents** - Include specialists like a ResearcherAgent or CriticAgent
3. **Trying different tasks** - Test with various complexity levels
4. **Adjusting manager parameters** - Tune `max_round_count`, `max_stall_count` for your use case
5. **Composing workflows** - Use `.as_agent()` to nest workflows within larger systems
6. **Switching models** - Compare performance across different model tiers

For more examples, see:
- `magentic_workflow_as_agent.py` - Detailed streaming example with researcher + coder
- `group_chat_workflow_as_agent.py` - Simpler group chat pattern
- Agent Framework documentation for advanced orchestration patterns